Build auto-encoder model

In [6]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np

encoding_dim = 200  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_seq = Input(shape=(5000,))
# "encoded" is the encoded representation of the input
#encoded = Dense(encoding_dim, activation='relu')(input_seq)
encoded = Dense(1024, activation='relu')(input_seq)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(1024, activation='relu')(decoded)
decoded = Dense(5000, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_seq, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_seq, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
#decoder_layers = autoencoder.layers[-5:-1]
# create the decoder model
#decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

print("ok")

Using TensorFlow backend.


ok


import library and load data

In [2]:
import numpy as np
import os
import nrrd
import glob
import math
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier

posintpaths= 'postrainsamples/postotalintensity.save'
negintpaths= 'negtrainsamples/negtotalintensity.save'


postotalintensity = np.load(posintpaths)
negtotalintensity = np.load(negintpaths)
#datapath = os.path.expanduser("~")+'/DATA/Case  '
print(postotalintensity.shape)
print(negtotalintensity.shape)

traindata=np.concatenate((postotalintensity[:800],negtotalintensity[:800]),axis=0)
testdata=np.concatenate((postotalintensity[800:],negtotalintensity[800:]),axis=0)
for i in range(len(traindata)):
    for j in range(5000):
        if traindata[i][j]>255:
            traindata[i][j]=255
for i in range(len(testdata)):
    for j in range(5000):
        if testdata[i][j]>255:
            testdata[i][j]=255
#normalize data to value between 0 to 1            
traindata=traindata.astype('float32') / 255.
testdata=testdata.astype('float32') / 255.

print(traindata.shape)
print(testdata.shape)
print("ok")

(875, 5000)
(879, 5000)
(1600, 5000)
(154, 5000)
ok


Train auto-encoder

In [7]:
autoencoder.fit(traindata, traindata,
                nb_epoch=200,
                batch_size=256,
                shuffle=True,
                validation_data=(testdata, testdata))

# encode and decode some digits
# note that we take them from the *test* set
print("ok")

Train on 1600 samples, validate on 151 samples
Epoch 1/200
1600/1600 [==============================] - 1s - loss: 0.6931 - val_loss: 0.6930
Epoch 2/200
1600/1600 [==============================] - 0s - loss: 0.6929 - val_loss: 0.6928
Epoch 3/200
1600/1600 [==============================] - 0s - loss: 0.6928 - val_loss: 0.6926
Epoch 4/200
1600/1600 [==============================] - 0s - loss: 0.6926 - val_loss: 0.6924
Epoch 5/200
1600/1600 [==============================] - 0s - loss: 0.6924 - val_loss: 0.6922
Epoch 6/200
1600/1600 [==============================] - 0s - loss: 0.6923 - val_loss: 0.6920
Epoch 7/200
1600/1600 [==============================] - 0s - loss: 0.6921 - val_loss: 0.6918
Epoch 8/200
1600/1600 [==============================] - 0s - loss: 0.6919 - val_loss: 0.6916
Epoch 9/200
1600/1600 [==============================] - 0s - loss: 0.6918 - val_loss: 0.6914
Epoch 10/200
1600/1600 [==============================] - 0s - loss: 0.6916 - val_loss: 0.6912
Epoch 11/200

In [9]:
#get encoder result
encoded_traindata = encoder.predict(traindata)
encoded_testdata = encoder.predict(testdata)
print(encoded_traindata.shape)
#make label for train and test data
poslabel=np.ones((800,), dtype=np.int)
neglabel=np.zeros((800,), dtype=np.int)
trainlabel=np.hstack((poslabel,neglabel))
poslabel=np.ones((72,), dtype=np.int)
neglabel=np.zeros((79,), dtype=np.int)
testlabel=np.hstack((poslabel,neglabel))
#build many kinds of classifier
svmclf = svm.SVC(kernel="rbf")
svmclf = svmclf.fit(encoded_traindata, trainlabel) 

rfclf = RandomForestClassifier(n_estimators=20)
rfclf = rfclf.fit(encoded_traindata, trainlabel)

abclf = AdaBoostClassifier(n_estimators=100)
abclf = abclf.fit(encoded_traindata, trainlabel)

gbclf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
gbclf = gbclf.fit(encoded_traindata, trainlabel)

bgclf = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)
bgclf = bgclf.fit(encoded_traindata, trainlabel)

gnbclf = GaussianNB()
gnbclf = gnbclf.fit(encoded_traindata, trainlabel)

print("OK")

(1600, 32)
OK


predict testdata ,give result and accuracy

In [11]:
#encode testdata
encoded_testdata = encoder.predict(testdata)
print(encoded_traindata.shape)
#predict testdata
rfresult=rfclf.predict(encoded_testdata)
abresult=abclf.predict(encoded_testdata)
gbresult=gbclf.predict(encoded_testdata)
svmresult=svmclf.predict(encoded_testdata)
bgresult=bgclf.predict(encoded_testdata)
gnbresult=gnbclf.predict(encoded_testdata)




#calculate pos,neg,total accuracy
rfposright=0
rfnegright=0
abposright=0
abnegright=0
gbnegright=0
gbposright=0
svmposright=0
svmnegright=0
bgposright=0
bgnegright=0
gnbnegright=0
gnbposright=0
for i in range(len(testlabel)):
    if testlabel[i]==rfresult[i] and testlabel[i]==1:
        rfposright+=1
    if testlabel[i]==rfresult[i] and testlabel[i]==0:
        rfnegright+=1
    if testlabel[i]==abresult[i] and testlabel[i]==1:
        abposright+=1
    if testlabel[i]==abresult[i] and testlabel[i]==0:
        abnegright+=1
    if testlabel[i]==gbresult[i] and testlabel[i]==1:
        gbposright+=1
    if testlabel[i]==gbresult[i] and testlabel[i]==0:
        gbnegright+=1
    if testlabel[i]==svmresult[i] and testlabel[i]==1:
        svmposright+=1
    if testlabel[i]==svmresult[i] and testlabel[i]==0:
        svmnegright+=1
    if testlabel[i]==bgresult[i] and testlabel[i]==1:
        bgposright+=1
    if testlabel[i]==bgresult[i] and testlabel[i]==0:
        bgnegright+=1
    if testlabel[i]==gnbresult[i] and testlabel[i]==1:
        gnbposright+=1
    if testlabel[i]==gnbresult[i] and testlabel[i]==0:
        gnbnegright+=1       
rftotalacu=(rfposright+rfnegright)/151
rfposright/=72
rfnegright/=79
abtotalacu=(abposright+abnegright)/151
abposright/=72
abnegright/=79
gbtotalacu=(gbposright+gbnegright)/151
gbposright/=72
gbnegright/=79
svmtotalacu=(svmposright+svmnegright)/151
svmposright/=72
svmnegright/=79
bgtotalacu=(bgposright+bgnegright)/151
bgposright/=72
bgnegright/=79
gnbtotalacu=(gnbposright+gnbnegright)/151
gnbposright/=72
gnbnegright/=79

print("Based on intensity with autoencoder---------------")

print("random forest:",rfposright,rfnegright,rftotalacu)
print("adaboost:",abposright,abnegright,abtotalacu)
print("gradient boost:",gbposright,gbnegright,gbtotalacu)
print("svm:",svmposright,svmnegright,svmtotalacu)
print("bagging:",bgposright,bgnegright,bgtotalacu)
print("gaussian naive bayes:",gnbposright,gnbnegright,gnbtotalacu)        
        
    
# rfindex=[i for i,x in enumerate(rfresult) if x==0]
# abindex=[i for i,x in enumerate(abresult) if x==0]
# gbindex=[i for i,x in enumerate(gbresult) if x==0]
# print("Based on intensity---------------")
# print("random forest:",rfresult,rfindex)
# print("adaboost:",abresult,abindex)
# print("gradient boost:",gbresult,gbindex)
#print("svm:",svmclf.predict(intensity))
#print("bagging:",bgclf.predict(intensity))
#print("gaussian naive bayes:",gnbclf.predict(intensity))
#print("SGD:",sgdclf.predict(intensity))

# rfacu=rfclf.score(encoded_testdata,testlabel)
# abacu=abclf.score(encoded_testdata,testlabel)
# gbacu=gbclf.score(encoded_testdata,testlabel)

(1600, 32)
0.960264900662 0.920529801325 0.913907284768
Based on intensity with autoencoder---------------
random forest: 0.9722222222222222 0.9493670886075949 0.9602649006622517
adaboost: 0.9722222222222222 0.8734177215189873 0.9205298013245033
gradient boost: 0.9305555555555556 0.8987341772151899 0.9139072847682119
svm: 0.9722222222222222 0.9873417721518988 0.9801324503311258
bagging: 0.9583333333333334 0.9873417721518988 0.9735099337748344
gaussian naive bayes: 0.9861111111111112 0.6075949367088608 0.7880794701986755


choose one case and get its intensity

In [13]:
import numpy as np
import os
import nrrd
import glob
import math
needlespath='case064vtk-auto.save'
nrrdpath = os.path.expanduser("~")+'/DATA/Case  064/NRRD/Manual_CY/16 AX space trial 1.6 SLICE COVER TEMPLATE.nrrd'
thistopath= 'needleintensity/totalhisto.save'
tmeanpath= 'needleintensity/totalhisto.save'

def print_full(x):
    np.set_printoptions(threshold=np.inf)
    print(x)
    np.set_printoptions(threshold=1000)
    
def str2float(list):
    newlist=[]
    n=list.ndim
    print(n)
    if n>1:       
        for j in range(len(list)):
            temp=[float(i) for i in list[j]]
            newlist.append(temp)
    else:
        newlist=[float(i) for i in list]
    return newlist    

def gettransmat(spacing,origin):
    spacing.append([0.,0.,0.])
    origin.append(1.)
    spacing=np.array(spacing)
    origin=np.array(origin)
    transmat=np.linalg.inv(np.c_[ spacing,origin ])
    return transmat

def transcoord(data,transmat):
    data=data.transpose()
    data=np.append(data,np.array([[1. for i in range(5000)]]),axis=0)
#    print(data,data.shape)
    
    newdata=np.dot(transmat,data)
    newdata=np.rint(newdata)
    newdata=newdata.astype(int)
#    print(newdata,newdata.shape)
    
    return newdata

def getintensity(imgneedles):
    meangray=[]
    intensity=[]
    rng=[-1,0,1]
    for i in range(len(imgneedles)):
        mean=0
        graylevel=[]
        for j in range(5000):
            temp=2000
            for x1 in rng:
                for x2 in rng:
                    if nrrddata[0][imgneedles[i][0][j]+x1][imgneedles[i][1][j]+x2][imgneedles[i][2][j]]<temp:
                        temp=nrrddata[0][imgneedles[i][0][j]+x1][imgneedles[i][1][j]+x2][imgneedles[i][2][j]]
            gray=temp
            graylevel.append(gray)
            mean+=gray
        mean/=5000
        meangray.append(mean)
        intensity.append(graylevel)
    return np.array(meangray),np.array(intensity)

def gethistogram(intensity):
    totalhis=[]
    for i in range(len(intensity)):
        upbound=10
        histogram=[]
        while upbound<155:
            num=sum(i < upbound and i>=(upbound-10) for i in intensity[i])/5000
            histogram.append(num)
            upbound+=10
        num=1-sum(histogram)
        histogram.append(num)
        totalhis.append(histogram)
        totalhis2=np.around(totalhis,2)
    return totalhis2   

def getsegmean(intensity):
    totalsegmean=[]
    for i in range(len(intensity)):
        upbound=500
        segmean=[]
        while upbound<=5000:
            num=np.mean(intensity[i][upbound-500:upbound])
            segmean.append(num)
            upbound+=500
        totalsegmean.append(segmean)
        totalsegmean2=np.around(totalsegmean,2)
    return totalsegmean2 

def sortseq(needles):
    newseq=[]
    for i in range(len(needles)):
        temp=needles[i][needles[i][:,2].argsort()]
        #print(temp)
        newseq.append(temp)
    return newseq    
totalhisto=np.load(thistopath)
averhisto=np.mean(totalhisto,axis=0)
needles = np.load(needlespath)
needles[:,:,:2]=needles[:,:,:2]*(-1.)
needles=sortseq(needles)
needles=np.array(needles)
nrrddata = nrrd.read(nrrdpath)
print(nrrddata[1])
print(nrrddata[1]['space origin'])  
nrrdspacing=str2float(np.array(nrrddata[1]['space directions']))
nrrdsporig=str2float(np.array(nrrddata[1]['space origin'])) 

transmat=gettransmat(nrrdspacing,nrrdsporig)

imgneedles=[]
for i in range(len(needles)):
    temp=transcoord(needles[i],transmat)
    imgneedles.append(temp)
    
mean,intensity=getintensity(imgneedles)
histo=gethistogram(intensity)
segmean=getsegmean(intensity)

for i in range(len(intensity)):
    for j in range(5000):
        if intensity[i][j]>255:
            intensity[i][j]=255
intensity=intensity.astype('float32') / 255.
print("ok")

{'type': 'unsigned short', 'endian': 'little', 'keyvaluepairs': {}, 'space': 'left-posterior-superior', 'encoding': 'gzip', 'dimension': 3, 'kinds': ['domain', 'domain', 'domain'], 'sizes': [640, 416, 120], 'space directions': [['0.46875', '0', '0'], ['0', '0.46875', '0'], ['0', '0', '1.600006103515625']], 'space origin': ['-185.93734741210935', '-105.98280334472663', '-106.57722473144517']}
['-185.93734741210935', '-105.98280334472663', '-106.57722473144517']
2
1
ok


predict real case and give result

In [23]:
encoded_testdata = encoder.predict(intensity)
print(encoded_traindata.shape)
#predict testdata
rfresult=rfclf.predict(encoded_testdata)
abresult=abclf.predict(encoded_testdata)
gbresult=gbclf.predict(encoded_testdata)
svmresult=svmclf.predict(encoded_testdata)
bgresult=bgclf.predict(encoded_testdata)
gnbresult=gnbclf.predict(encoded_testdata)

rfindex=[i for i,x in enumerate(rfresult) if x==0]

for i in range(40):
    print(i,i not in rfindex,histo[i])


0 True [ 0.78  0.06  0.05  0.04  0.02  0.02  0.01  0.01  0.    0.    0.    0.    0.
  0.    0.    0.  ]
1 True [ 0.81  0.08  0.03  0.02  0.02  0.01  0.02  0.01  0.    0.    0.    0.    0.
  0.    0.    0.  ]
2 False [ 0.23  0.04  0.28  0.11  0.06  0.05  0.03  0.03  0.04  0.03  0.04  0.02
  0.01  0.    0.01  0.02]
3 True [ 0.72  0.1   0.05  0.04  0.03  0.01  0.02  0.01  0.01  0.    0.    0.    0.
  0.    0.    0.  ]
4 True [ 0.74  0.08  0.04  0.05  0.02  0.02  0.01  0.02  0.01  0.01  0.01  0.01
  0.    0.    0.    0.  ]
5 True [ 0.72  0.08  0.05  0.04  0.03  0.03  0.01  0.01  0.    0.01  0.01  0.    0.
  0.    0.    0.  ]
6 True [ 0.65  0.12  0.05  0.06  0.04  0.02  0.02  0.02  0.01  0.01  0.01  0.    0.
  0.    0.    0.01]
7 True [ 0.73  0.09  0.05  0.03  0.01  0.01  0.01  0.01  0.02  0.01  0.    0.01
  0.    0.01  0.    0.02]
8 False [ 0.6   0.13  0.09  0.06  0.04  0.03  0.02  0.01  0.01  0.    0.01  0.    0.
  0.    0.    0.  ]
9 False [ 0.54  0.12  0.08  0.06  0.06  0.04  0.05  0.03

build a neural network for training encoded intensity

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(64, input_dim=32, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(encoded_traindata, trainlabel,
          nb_epoch=200,
          batch_size=16)

# result=model.predict(intensity)
# print(result)
#score = model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/200
1600/1600 [==============================] - 0s - loss: 0.4833 - acc: 0.7488     
Epoch 2/200
1600/1600 [==============================] - 0s - loss: 0.4010 - acc: 0.8056     
Epoch 3/200
1600/1600 [==============================] - 0s - loss: 0.3760 - acc: 0.8187     
Epoch 4/200
1600/1600 [==============================] - 0s - loss: 0.3438 - acc: 0.8450     
Epoch 5/200
1600/1600 [==============================] - 0s - loss: 0.3478 - acc: 0.8431     
Epoch 6/200
1600/1600 [==============================] - 0s - loss: 0.3259 - acc: 0.8506     
Epoch 7/200
1600/1600 [==============================] - 0s - loss: 0.3097 - acc: 0.8631     
Epoch 8/200
1600/1600 [==============================] - 0s - loss: 0.3138 - acc: 0.8600     
Epoch 9/200
1600/1600 [==============================] - 0s - loss: 0.3081 - acc: 0.8662     
Epoch 10/200
1600/1600 [==============================] - 0s - loss: 0.3063 - acc: 0.8625     
Epoch 11/200
1600/1600 [==============================] - 0

show the result

In [11]:
model.evaluate(encoded_testdata, testlabel, batch_size=32)

 32/151 [=====>........................] - ETA: 0s

[0.18818191364900955, 0.9403973525723085]

In [14]:
result=model.predict(encoded_testdata)
result=[1 if x>0.5 else 0 for x in result]
nnposright=0
nnnegright=0
for i in range(len(testlabel)):
    if testlabel[i]==result[i] and testlabel[i]==1:
        nnposright+=1
    if testlabel[i]==result[i] and testlabel[i]==0:
        nnnegright+=1
totalacu=(nnposright+nnnegright)/151
nnposright/=72
nnnegright/=79
print(nnposright,nnnegright,totalacu)

0.9166666666666666 0.8607594936708861 0.8874172185430463


build a neural network for training intensity

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(1024, input_dim=5000, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(traindata, trainlabel,
          nb_epoch=200,
          batch_size=16)

Epoch 1/200
1600/1600 [==============================] - 2s - loss: 1.0659 - acc: 0.5694     
Epoch 2/200
1600/1600 [==============================] - 2s - loss: 0.6513 - acc: 0.6950     
Epoch 3/200
1600/1600 [==============================] - 2s - loss: 0.4879 - acc: 0.7950     
Epoch 4/200
1600/1600 [==============================] - 2s - loss: 0.4402 - acc: 0.8144     
Epoch 5/200
1600/1600 [==============================] - 2s - loss: 0.4104 - acc: 0.8444     
Epoch 6/200
1600/1600 [==============================] - 2s - loss: 0.3834 - acc: 0.8538     
Epoch 7/200
1600/1600 [==============================] - 2s - loss: 0.3763 - acc: 0.8731     
Epoch 8/200
1600/1600 [==============================] - 2s - loss: 0.3427 - acc: 0.8881     
Epoch 9/200
1600/1600 [==============================] - 2s - loss: 0.3193 - acc: 0.9044     
Epoch 10/200
1600/1600 [==============================] - 2s - loss: 0.3519 - acc: 0.9094     
Epoch 11/200
1600/1600 [==============================] - 2

In [16]:
model.evaluate(testdata, testlabel)

128/151 [========================>.....] - ETA: 0s

[2.0145814118795835, 0.86092715705467371]

In [16]:
result=model.predict(testdata)
result=[1 if x>0.5 else 0 for x in result]
nnposright=0
nnnegright=0
for i in range(len(testlabel)):
    if testlabel[i]==result[i] and testlabel[i]==1:
        nnposright+=1
    if testlabel[i]==result[i] and testlabel[i]==0:
        nnnegright+=1
totalacu=(nnposright+nnnegright)/151
nnposright/=72
nnnegright/=79
print(result)
print(nnposright,nnnegright,totalacu)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
0.9166666666666666 0.7721518987341772 0.8410596026490066
